In [17]:
import pandas as pd
import numpy as np
import requests
import time

In [18]:
apikey = "LR691R2UFT21EU6C"

In [19]:
companies = ['AAPL']
indicators=['SMA', 'EMA','STOCH','RSI',
       'ADX', 'CCI', 'AROON', 'BBANDS', 'AD', 'OBV']
time_period_independent = ['STOCH','AD','OBV', 'MACD']
interval="daily"

In [20]:
# SMA 10, 20
# stoch, AD, OBV

In [21]:
# obtain csv of daily stock info
def get_daily(company):
    url="https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey={}".format(company, apikey)
    response = requests.get(url)
    data = response.json()['Time Series (Daily)']
    daily = pd.DataFrame(data).T
    daily.to_csv('./data/daily_{}.csv'.format(company), index = True)

In [22]:
def get_indicator_data(indicator, company, interval="daily",series_type="open",time_period = 0):
    if time_period == 0:
        indicator_url = "https://www.alphavantage.co/query?function={}&symbol={}&interval={}&series_type={}&apikey={}".format(indicator, company, interval,series_type, apikey)
    else:
        indicator_url = "https://www.alphavantage.co/query?function={}&symbol={}&interval={}&time_period={}&series_type={}&apikey={}".format(indicator, company, interval,time_period,series_type, apikey)
    response=requests.get(indicator_url)
    result = response.json()
    column = list(result.keys())
    if len(column) > 1:
        choice = list(result.keys())[1]
        data = pd.DataFrame(result[choice])
        return data
    else:
        print(column, indicator_url)
        return None

In [23]:
time.sleep(60)

In [24]:
for company in companies:
    get_daily(company)
    
    open_ind = pd.DataFrame()
    close_ind = pd.DataFrame()
    high_ind = pd.DataFrame()
    low_ind = pd.DataFrame()
    for indicator in indicators:
        if indicator in time_period_independent:
            open = get_indicator_data(indicator, company,interval,"open")
            close = get_indicator_data(indicator, company,interval,"close")
            high = get_indicator_data(indicator, company,interval,"high")
            low = get_indicator_data(indicator, company,interval,"low")
            open_ind = pd.concat([open_ind,open])
            close_ind = pd.concat([close_ind,close])
            high_ind = pd.concat([high_ind,high])
            low_ind = pd.concat([low_ind,low])
            time.sleep(60)
            
        else:
            for time_period in [10,20]:
                open = get_indicator_data(indicator, company,interval,"open",time_period)
                close = get_indicator_data(indicator, company,interval,"close",time_period)
                high = get_indicator_data(indicator, company,interval,"high",time_period)
                low = get_indicator_data(indicator, company,interval,"low",time_period)
            
                open['indicator'] = (open.index+'-'+str(time_period))
                open = open.set_index('indicator')
                close['indicator'] = (open.index+'-'+str(time_period))
                close = close.set_index('indicator')
                high['indicator'] = (open.index+'-'+str(time_period))
                high = high.set_index('indicator')
                low['indicator'] = (open.index+'-'+str(time_period))
                low = low.set_index('indicator')

                open_ind = pd.concat([open_ind,open])
                close_ind = pd.concat([close_ind,close])
                high_ind = pd.concat([high_ind,high])
                low_ind = pd.concat([low_ind,low])
                time.sleep(60)

    open_ind = open_ind.drop_duplicates().T
    close_ind = close_ind.drop_duplicates().T
    high_ind = high_ind.drop_duplicates().T
    low_ind = low_ind.drop_duplicates().T

    open_ind.to_csv('./data/open_ind_{}.csv'.format(company), index = False)
    close_ind.to_csv('./data/close_ind_{}.csv'.format(company), index = False)
    high_ind.to_csv('./data/high_ind_{}.csv'.format(company), index = False)
    low_ind.to_csv('./data/low_ind_{}.csv'.format(company), index = False)